In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
import cv2
from PIL import Image
import math
import scipy
from scipy import stats
%matplotlib notebook

In [68]:
class Code_rec():
    # 用cv2转成HSV颜色空间，轻松提取不同色调的字符str
    def extract_image(self, four_str_color, im_hsv):
        image_list = []
        for i in four_str_color:
            lower = np.array([i, 0, 0])
            upper = np.array([i, 255, 255])
            mask = cv2.inRange(im_hsv,lowerb=lower, upperb=upper)
            image_list.append(mask)
        return image_list

    #用递归计算叠加最大值（其实有点小题大做了）——封装好了，以后只管调用就好，懒得再看了，也不写备注了
    def calculate_index(self, lst):
        ll = []
        def foo(lst, i):
            try:
                if lst[i] == 0:
                    return 0
                else:
                    x = lst[i] + foo(lst, i+1)
                    return x
            except:
                return lst[i]
        for i in range(len(lst)):
            ll.append(foo(lst, i))
#         print(ll)
        #最大的索引，也就是字符出现的最左边的索引位置
        left_index = np.argmax(ll)
#         print("left_index", left_index)
        for n, i in enumerate(ll[left_index:]):
            if i == 0:
                length = n
                break
#         print("length:", length)
        try:
#             print("cao ni ma")
            length
        except:
            print("*"*80, "this is length error")
            length = 120-left_index
        
        mid_index = left_index + length // 2
#         mid_index = left_index + 20
#         print("mid_index:", mid_index)
        #返回每张img的中间位置索引
        return mid_index
    
         
    def get_img_order_index(self, imgs):
        """
        得到提取出来的4个不同颜色的img列表，计算他们的排列顺序、中心位置，并返回
        """
        count = 0
        wrapper_dict = {}
        for num, img in enumerate(imgs):
            pixel_num_lst = []
            for index, col in enumerate(img.T):
                pixel_num = 0
                for row in col:
                    count += 1
                    if row == 255:
                        pixel_num += 1
                pixel_num_lst.append(pixel_num)
            
            mid_index = self.calculate_index(pixel_num_lst)  # 返回字符的左右边界的索引
#             print("---------------mid_index:", mid_index)
            wrapper_dict[num] = mid_index
#         print(wrapper_dict)
        order_index = np.argsort([mid for mid in wrapper_dict.values()]) # 得到从左到右的排序的图片索引
        ordered_imgs = [j for j in  zip(np.array(imgs)[order_index], [wrapper_dict[i] for i in order_index])]
        #上面ordered_imgs 的返回结构：[(<im1>, mid_index), (<im2>, mid_index)...]
#         print(ordered_imgs)
        return ordered_imgs

    def cut_image(self, imgs):
        """
        按中间位置左右切割20个像素，得到一个小正方形的str二值图案
        """
        img_cuted_lst = []
        for i in imgs:
            img_cuted = i[0][:, i[1]-20:i[1]+20]
#             print(img_cuted)
            img_cuted_lst.append(img_cuted)
        return img_cuted_lst
        
    def save_image(self, imgs, filename=None):
        num = 1
        if not filename:
            for img in imgs:
                cv2.imwrite("./code_imgs/img%s.jpg"%num, img)
                num += 1
        else:
            for img in imgs:
                cv2.imwrite("./code_imgs/img%s--%s.jpg"%(filename,num), img)
                num += 1
        
    def convert_image(self, filename):
        """
        输入：原始的验证码的文件路径名
        输出：分割好的 4 张二值化的小正方形的img矩阵列表  
            （format: [im1, im2, im3, im4] ）
             (use: 1.存储 2.训练模型 3.喂给模型得到预测结果 )
        
        """
        im = cv2.imread(filename)
#         cv2.imshow("im-", im)
        im = cv2.morphologyEx(im, cv2.MORPH_CLOSE, (2, 1))
        im = cv2.medianBlur(im, 3)

        im_hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)#把BGR图像转换为HSV格式
        im_bins = np.bincount(im_hsv.reshape(-1, 3)[:, 0])
        four_str_color = np.argsort(im_bins)[-1::-1][1:5]

        image_extract_list = self.extract_image(four_str_color, im_hsv)
        img_order_lst = self.get_img_order_index(image_extract_list)  # 得到img的从左到右的排列顺序，以及每个img的left_right索引，等待切割
        img_converted_lst = self.cut_image(img_order_lst)
#         print("$$"*80)
#         self.save_image(img_cuted_lst)
        
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
        return img_converted_lst
    

In [ ]:
# 爬取1000张验证码截图
import requests

headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.67 Safari/537.36",
    # "Host":"hz.lianjia.com",
    "Referer" : "https://hz.lianjia.com/zufang/rs/"
}
proxies = {
    "http" :"http://localhost:1080",
    "https" :"https://localhost:1080"
}
for i in range(1, 2000):
    r = requests.get("https://upassport.lianjia.com/freshCaptch?t=1542017409165", headers=headers, proxies=proxies)
    with open("./xxx/t%s.jpg"%i, "wb") as f:
        f.write(r.content)



In [ ]:
# 对验证码图片进行大小切割

for i in range(1, 1000):
    im = cv2.imread("./code_img/t%s.jpg"%i)
    im = im[:40, :120, :]
#     print(im.shape)
    cv2.imwrite("./code_img/t%s.jpg"%i, im)

In [71]:
#放进写好的类里，提取出二值化图
code_rec = Code_rec()
for i in range(1, 1000):
    imgs = code_rec.convert_image("./xxx/t%s.jpg"%i)
    code_rec.save_image(imgs, "%s"%i)
    print("i: ", i)
print("finished!!!")

i:  1
i:  2
i:  3
i:  4
i:  5
i:  6
i:  7
i:  8
i:  9
i:  10
i:  11
i:  12
i:  13
i:  14
i:  15
i:  16
i:  17
i:  18
i:  19
i:  20
i:  21
i:  22
i:  23
i:  24
i:  25
i:  26
i:  27
i:  28
i:  29
i:  30
i:  31
i:  32
i:  33
i:  34
i:  35
i:  36
i:  37
i:  38
i:  39
i:  40
i:  41
i:  42
i:  43
i:  44
i:  45
i:  46
i:  47
i:  48
i:  49
i:  50
i:  51
i:  52
i:  53
i:  54
i:  55
i:  56
i:  57
i:  58
i:  59
i:  60
i:  61
i:  62
i:  63
i:  64
i:  65
i:  66
i:  67
i:  68
i:  69
i:  70
i:  71
i:  72
i:  73
i:  74
i:  75
i:  76
i:  77
i:  78
i:  79
i:  80
i:  81
i:  82
i:  83
i:  84
i:  85
i:  86
i:  87
i:  88
i:  89
i:  90
i:  91
i:  92
i:  93
i:  94
i:  95
i:  96
i:  97
i:  98
i:  99
i:  100
i:  101
i:  102
i:  103
i:  104
i:  105
i:  106
i:  107
i:  108
i:  109
i:  110
i:  111
i:  112
i:  113
i:  114
i:  115
i:  116
i:  117
i:  118
i:  119
i:  120
i:  121
i:  122
i:  123
i:  124
i:  125
i:  126
i:  127
i:  128
i:  129
i:  130
i:  131
i:  132
i:  133
i:  134
i:  135
i:  136
i:  137
i:  138
i:  